# Introduction

In this notebook, we will do a comprehensive analysis of the Android app market by comparing thousands of apps in the Google Play store.

# About the Dataset of Google Play Store Apps & Reviews

**Data Source:** <br>
App and review data was scraped from the Google Play Store by Lavanya Gupta in 2018. Original files listed [here](
https://www.kaggle.com/lava18/google-play-store-apps).

# Import Statements

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# Notebook Presentation

In [ ]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = '{:,.2f}'.format

# Read the Dataset

In [ ]:
df_apps = pd.read_csv('apps.csv')

# Data Cleaning

**Challenge**: How many rows and columns does `df_apps` have? What are the column names? Look at a random sample of 5 different rows with [.sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html).

In [ ]:
# get 5 random selections to see data of dataframe
df_apps.sample(n=5)

In [ ]:
df_apps.shape

In [ ]:
df_apps.columns

### Drop Unused Columns

**Challenge**: Remove the columns called `Last_Updated` and `Android_Version` from the DataFrame. We will not use these columns. 

In [ ]:
# drop two unnecessary columns
df_apps.drop(axis=1, labels=['Last_Updated', 'Android_Ver'])

### Find and Remove NaN values in Ratings

**Challenge**: How may rows have a NaN value (not-a-number) in the Ratings column? Create DataFrame called `df_apps_clean` that does not include these rows. 

In [ ]:
# check how many null values in 'Rating' column and count them
df_apps['Rating'].isna().sum()

# an alternate solution if you want to check the null values in the
# corresponding column of your choice, but still see the whole table
# containing those rows is this....
nan_rows = df_apps[df_apps.Rating.isna()]
print(nan_rows.shape)
nan_rows.head()

In [ ]:
df_apps_clean = df_apps.dropna()

In [ ]:
df_apps_clean.shape

### Find and Remove Duplicates

**Challenge**: Are there any duplicates in data? Check for duplicates using the [.duplicated()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) function. How many entries can you find for the "Instagram" app? Use [.drop_duplicates()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) to remove any duplicates from `df_apps_clean`. 


In [ ]:
# the sum function, when used with a finder function like duplicated,
# counts the values that corresponds to True.

# so when used with these types of functions, it then serves as a counter
# for values that satisfy the function's conditions.
df_apps_clean.duplicated().sum()

In [ ]:
df_apps_clean = df_apps_clean.drop_duplicates(subset=['App', 'Type', 'Price'])

In [ ]:
df_apps_clean[df_apps_clean.App == 'Instagram']

# Find Highest Rated Apps

**Challenge**: Identify which apps are the highest rated. What problem might you encounter if you rely exclusively on ratings alone to determine the quality of an app?

In [ ]:
df_apps_clean.sort_values(by='Rating', ascending=False).head()

# Find 5 Largest Apps in terms of Size (MBs)

**Challenge**: What's the size in megabytes (MB) of the largest Android apps in the Google Play Store. Based on the data, do you think there could be limit in place or can developers make apps as large as they please? 

In [ ]:
df_apps_clean.sort_values(by='Size_MBs', ascending=False).head()

# Find the 5 App with Most Reviews

**Challenge**: Which apps have the highest number of reviews? Are there any paid apps among the top 50?

In [ ]:
df_apps_clean.sort_values(by='Reviews', ascending=False).head()

# Plotly Pie and Donut Charts - Visualise Categorical Data: Content Ratings

In [ ]:
ratings = df_apps_clean.Content_Rating.value_counts()
ratings

In [ ]:
fig = px.pie(labels=ratings.index, values=ratings.values, title='Content Rating', names=ratings.index)

fig.show()

In [ ]:
fig = px.pie(labels=ratings.index, values=ratings.values, title='Content Rating', names=ratings.index)
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

In [ ]:
fig = px.pie(labels=ratings.index, values=ratings.values, title='Content Rating', names=ratings.index, hole=0.6)
fig.update_traces(textposition='inside', textfont_size=15, textinfo='percent')
fig.show()

# Numeric Type Conversion: Examine the Number of Installs

**Challenge**: How many apps had over 1 billion (that's right - BILLION) installations? How many apps just had a single install? 

Check the datatype of the Installs column.

Count the number of apps at each level of installations. 

Convert the number of installations (the Installs column) to a numeric data type. Hint: this is a 2-step process. You'll have make sure you remove non-numeric characters first. 

In [ ]:
# get count of installations
df_apps_clean.sort_values(by='Installs', ascending=False).Installs.value_counts()

In [ ]:
# check non-numeric values in 'Installs' column
df_apps_clean.Installs = df_apps_clean.Installs.astype(str).str.replace(',', '')
df_apps_clean.Installs = pd.to_numeric(df_apps_clean.Installs)
df_apps_clean[['App', 'Installs']].groupby('Installs').count()

# Find the Most Expensive Apps, Filter out the Junk, and Calculate a (ballpark) Sales Revenue Estimate

Let's examine the Price column more closely.

**Challenge**: Convert the price column to numeric data. Then investigate the top 20 most expensive apps in the dataset.

Remove all apps that cost more than $250 from the `df_apps_clean` DataFrame.

Add a column called 'Revenue_Estimate' to the DataFrame. This column should hold the price of the app times the number of installs. What are the top 10 highest grossing paid apps according to this estimate? Out of the top 10 highest grossing paid apps, how many are games?


In [ ]:
# convert 'Price' column from string to float
df_apps_clean.Price = df_apps_clean.Price.astype(str).str.replace('$', '', regex=True)
df_apps_clean.Price = pd.to_numeric(df_apps_clean.Price)
df_apps_clean.sort_values(by='Price', ascending=False).head()

### The most expensive apps sub $250

In [ ]:
# remove apps that are overpriced, since they appear to be suspicious and could manipulate our data
df_apps_clean = df_apps_clean.drop(df_apps_clean[df_apps_clean.Price > 250].index)
df_apps_clean.sort_values(by='Price', ascending=False).head(10)

### Highest Grossing Paid Apps (ballpark estimate)

In [ ]:
# highest paid apps
df_apps_clean['Revenue_Estimate'] = df_apps_clean['Installs'] * df_apps_clean.Price
df_apps_clean.sort_values(by='Revenue_Estimate', ascending=False).head(10)

# Plotly Bar Charts & Scatter Plots: Analysing App Categories

In [ ]:
# the nunique method returns the number of unique inputs in a column
df_apps_clean.Category.nunique()

In [ ]:
# slice the dataframe the value_counts method returns from counting the number of occurrences 
# a given input has in the column from indices 0 up to 9
top10_category = df_apps_clean.Category.value_counts()[:10]
top10_category

### Vertical Bar Chart - Highest Competition (Number of Apps)

In [ ]:
bar = px.bar(
    x=top10_category.index,     # the category name 
    y=top10_category.values)    # the value corresponding to the category

bar.show()

### Horizontal Bar Chart - Most Popular Categories (Highest Downloads)

In [ ]:
# the agg method works by chaining commands we want as a column in our new grouped dataframe

# in this example, the dataframe first gets grouped according to category, which would show
# the rows with groups all sorted together.

# the agg method then, when used, takes as index the column argument in the groupby method,
# and makes a new column based on the dict values that is passed. In our example, the 'Installs'
# column is made by adding all the Install values of every app in a given category.

# the process of passing a dict can contain many columns in which to do different methods for
# relevant data handling
category_installs = df_apps_clean.groupby('Category').agg({'Installs': pd.Series.sum})
category_installs.sort_values(by='Installs', ascending=True, inplace=True)

category_installs.head(10)

In [ ]:
h_bar = px.bar(
    x = category_installs.Installs,
    y = category_installs.index,
    orientation = 'h'       # indicates horizontal bar
)

h_bar.show()

### Category Concentration - Downloads vs. Competition

**Challenge**: 
* First, create a DataFrame that has the number of apps in one column and the number of installs in another:

<img src=https://imgur.com/uQRSlXi.png width="350">

* Then use the [plotly express examples from the documentation](https://plotly.com/python/line-and-scatter/) alongside the [.scatter() API reference](https://plotly.com/python-api-reference/generated/plotly.express.scatter.html)to create scatter plot that looks like this. 

<img src=https://imgur.com/cHsqh6a.png>

*Hint*: Use the size, hover_name and color parameters in .scatter(). To scale the yaxis, call .update_layout() and specify that the yaxis should be on a log-scale like so: yaxis=dict(type='log') 

In [ ]:
# # use agg method to chain different method calls for the groupby function
# category_app = df_apps_clean.groupby('Category').agg({'Installs': pd.Series.sum, 'App': pd.Series.count})
# category_app.sort_values(by='Installs', ascending=False, inplace=True)
# category_app.head()

# alternatively, we can use the merge function to make use of our previously made categorized dataframe
cat_number = df_apps_clean.groupby('Category').agg({'App': pd.Series.count})
cat_merged_df = pd.merge(cat_number, category_installs, on='Category', how='inner')
cat_merged_df.sort_values('Installs', ascending=False, inplace=True)

cat_merged_df.head()

In [ ]:
# this nifty function from plotly makes a scatter plot with just a few arguments
# this will take a lot of time to learn, but will be very useful for a data scientist
plot = px.scatter(
    data_frame = cat_merged_df,
    x = "App",
    y = "Installs",

    # the key depends on the values of the x and y
    labels = {'App': 'Number of Apps (Lower=More concentrated)'},

    color = 'Installs',
    title = 'Category Concentration',
    hover_name = cat_merged_df.index,
    size = 'App',
)

plot.update_layout(yaxis=dict(type='log'))

plot.show()

# Extracting Nested Data from a Column

**Challenge**: How many different types of genres are there? Can an app belong to more than one genre? Check what happens when you use .value_counts() on a column with nested values? See if you can work around this problem by using the .split() function and the DataFrame's [.stack() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html). 


In [ ]:
# so, to get this straight, pandas has their own configured python string methods that
# can be used to alter the data in the dataframe. like this str.split method used in a
# Series datatype.

# I don't really understand how the stack works, but I wish it gets explained soon.
stack = df_apps_clean.Genres.str.split(';', expand=True).stack()
num_genres = stack.value_counts()

num_genres

# Colour Scales in Plotly Charts - Competition in Genres

**Challenge**: Can you create this chart with the Series containing the genre data? 

<img src=https://imgur.com/DbcoQli.png width=400>

Try experimenting with the built in colour scales in Plotly. You can find a full list [here](https://plotly.com/python/builtin-colorscales/). 

* Find a way to set the colour scale using the color_continuous_scale parameter. 
* Find a way to make the color axis disappear by using coloraxis_showscale. 

In [ ]:
genre_bar = px.bar(
    x=num_genres.index[:15],
    y=num_genres.values[:15],
    title='Top Genres',
    labels={'x': 'Genre', 'y': 'Number of Apps'},
    hover_name=num_genres.index[:15],
    color_continuous_scale='Gnbu'
)

genre_bar.update_layout(
    coloraxis_showscale=False
)

genre_bar.show()

# Grouped Bar Charts: Free vs. Paid Apps per Category

In [ ]:
df_apps_clean.Type.value_counts()

In [ ]:
df_free_vs_paid = df_apps_clean.groupby(['Category', 'Type'], as_index=False).agg({'App': pd.Series.count})
df_free_vs_paid.head()

**Challenge**: Use the plotly express bar [chart examples](https://plotly.com/python/bar-charts/#bar-chart-with-sorted-or-ordered-categories) and the [.bar() API reference](https://plotly.com/python-api-reference/generated/plotly.express.bar.html#plotly.express.bar) to create a bar chart: 

You'll want to use the `df_free_vs_paid` DataFrame that you created above that has the total number of free and paid apps per category. 

See if you can figure out how to get the look above by changing the `categoryorder` to 'total descending' as outlined in the documentation here [here](https://plotly.com/python/categorical-axes/#automatically-sorting-categories-by-name-or-total-value). 

In [47]:
group_bar = px.bar(
    data_frame=df_free_vs_paid,
    x='Category',
    y='App',

    # the two arguments below work in unison; meaning that without the other, the graph
    # wouldn't show as expected.
    color='Type',
    barmode='group'     # specify that you want the graph to show as group
)

group_bar.update_layout(
    yaxis_title='Number of Apps',
    yaxis=dict(type='log'),
    xaxis=dict(categoryorder='total descending'),       # to arrange the graph descending
    title='Free vs Paid by Category'
)

group_bar.show()

# Plotly Box Plots: Lost Downloads for Paid Apps

**Challenge**: Create a box plot that shows the number of Installs for free versus paid apps. How does the median number of installations compare? Is the difference large or small?

Use the [Box Plots Guide](https://plotly.com/python/box-plots/) and the [.box API reference](https://plotly.com/python-api-reference/generated/plotly.express.box.html) to create a box chart. 


In [62]:
fvp_box = px.box(
    data_frame=df_apps_clean,
    x='Type',
    y='Installs',
    color='Type',
    notched=True,
    points='all'
)

fvp_box.update_layout(
    yaxis=dict(type='log'),
    title='How Many Downloads are Paid Apps Giving Up?'
)

fvp_box.show()

# Plotly Box Plots: Revenue by App Category

**Challenge**: See if you can generate a box chart below: 

Looking at the hover text, how much does the median app earn in the Tools category? If developing an Android app costs $30,000 or thereabouts, does the average photography app recoup its development costs?

Hint: I've used 'min ascending' to sort the categories. 

In [84]:
df_paid_apps = df_apps_clean[df_apps_clean.Type =='Paid']

revenue_box = px.box(
    x=df_paid_apps['Category'],
    y=df_paid_apps['Revenue_Estimate'],
    title='How Much Can Paid Apps Earn?'
)

revenue_box.update_layout(
    yaxis_title='Paid App Ballpark Revenue',
    xaxis_title='Category',
    yaxis=dict(type='log'),
    xaxis=dict(categoryorder='min ascending')
)

revenue_box.show()

# How Much Can You Charge? Examine Paid App Pricing Strategies by Category

**Challenge**: What is the median price price for a paid app? Then compare pricing by category by creating another box plot. But this time examine the prices (instead of the revenue estimates) of the paid apps. I recommend using `{categoryorder':'max descending'}` to sort the categories.

In [85]:
df_paid_apps.Price.median()

2.99

In [78]:
price_box = px.box(
    x=df_apps_clean[df_apps_clean.Type == 'Paid']['Category'],
    y=df_apps_clean[df_apps_clean.Type == 'Paid']['Price'],
    title='What is the Median Price for a Paid App?'
)

price_box.update_layout(
    yaxis_title='Price',
    xaxis_title='Category',
    yaxis=dict(type='log'),
    xaxis=dict(categoryorder='max descending')
)

price_box.show()